In [ ]:
import pandas as pd
# Plow that show the distribution of the number of items per session
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_ecommerce= pd.read_json('1_ecommerce.jsonl', lines=True)
df_ecommerce.head(5)

In [ ]:
# for each session, we will create a list of items that the user has clicked on, removing duplicates
events_list = []
for i in df_ecommerce.events:
    clicks = []
    for j in i:
        if j['type'] == 'clicks':
            clicks.append(j['aid'])
    # remove duplicates from clicks
    # clicks = set(clicks)
    events_list.append(list(clicks))

In [ ]:
df_ecommerce['items'] = events_list
df = df_ecommerce.drop(columns=["events"], axis=1)

In [ ]:
df.head()

In [ ]:
# takes only the session with items length higher than 20
df_truncated = df[df['items'].apply(lambda x: len(x) > 20)]
df_truncated


In [ ]:
df= df_truncated.copy()
# redefine index    
df.reset_index(drop=True, inplace=True)
df

In [ ]:
# Explode the 'items' column to create one row per item per session
df_exploded_items = df.explode('items')

In [ ]:
df_exploded_items

In [ ]:
# count how many times each item appears in the dataset
item_counts = df_exploded_items['items'].value_counts()
item_counts

In [ ]:
# create a plot of the distribution of the number of each item in the dataset
plt.figure(figsize=(10, 6))
plt.hist(item_counts, bins=500, color='blue', alpha=0.7)
plt.title('Distribution of the Number of Items per Session')
plt.xlabel('Number of Items')
plt.ylabel('Frequency')
# set x-axis limits to 0-100
plt.xlim(0, 100)
plt.grid(True)
plt.show()

In [ ]:
# remove all the items that appear less than 10 times in the dataset
item_counts = item_counts[item_counts > 10]

# re,pve each row in the dataset that contains an item that appears less than 10 times in the dataset
df_exploded_items = df_exploded_items[df_exploded_items['items'].isin(item_counts.index)]

In [ ]:
# Count the occurrences of each item in each session and drop duplicates
df_exploded_items['item_count'] = df_exploded_items.groupby(['session', 'items'])['items'].transform('count')
df_exploded_items = df_exploded_items.drop_duplicates(subset=['session', 'items'])



In [ ]:
# cerca un valore che ha item_count > 1
df_exploded_items[df_exploded_items['item_count'] > 1].sort_values(by='item_count', ascending=False).head(10)


In [ ]:
plt.figure(figsize=(12, 6))
# Set the title of the plot
plt.title('Distribution of the number of items per session')
# Set the x and y labels
plt.xlabel('Number of items')
plt.ylabel('Frequency')
# Create the histogram
plt.hist(df_exploded_items['item_count'], bins=50, color='blue', alpha=0.7)
# Show the plot
plt.show()

In [ ]:
# work only with the first 1000 sessions of the dataset
#df_exploded_items = df_exploded_items.head(100_000)
#df_exploded_items = df_exploded_items.reset_index(drop=True)
df_exploded_items

In [ ]:
# transform the itemCount in a float number
#df_exploded_items['item_count'] = df_exploded_items['item_count'].astype(float)

In [ ]:
# use the index of the dataframe as session id
#df_exploded_items['session'] = df_exploded_items.index
df_exploded_items['items'] = df_exploded_items['items'].astype('category')
df_exploded_items


In [ ]:
df_pivot = df_exploded_items.pivot_table(index='session', columns='items', values='item_count', aggfunc='first')
df_pivot

In [ ]:
import numpy as np
import pandas as pd

# Supponiamo che df_pivot sia già stato creato come segue:
# df_pivot = df_exploded_items.pivot_table(index='session', columns='items', values='item_count', aggfunc='sum')
# Per questa implementazione, consideriamo il campo item_count come rating

# Convertiamo il DataFrame in una matrice numpy, mantenendo i NaN per le celle mancanti
R_df = df_pivot.copy()
# Creiamo una maschera: 1 per gli elementi osservati, 0 altrimenti
mask = (~R_df.isna()).astype(float).values

# Sostituiamo i NaN in R con 0 (non verranno usati nel calcolo dell'errore grazie alla maschera)
R = R_df.fillna(0).values

def unconstrained_matrix_factorization(R, mask, k=10, epochs=100, alpha=0.001):
    """
    R: matrice dei rating (numpy array) ottenuta dal pivot, con 0 per le celle mancanti
    mask: matrice con 1 per gli elementi osservati e 0 per i mancanti
    k: numero di fattori latenti
    epochs: numero di iterazioni
    alpha: learning rate
    """
    num_users, num_items = R.shape
    # Inizializzazione casuale delle matrici dei fattori
    U = np.random.rand(num_users, k)
    V = np.random.rand(num_items, k)
    
    losses = []  # per tenere traccia della funzione di costo

    for epoch in range(epochs):
        # Calcolo delle predizioni
        R_hat = U.dot(V.T)
        # Calcolo dell'errore solo sugli elementi osservati
        error = (R - R_hat) * mask
        # Calcolo della funzione di costo (loss)
        cost = np.sum(error**2) / 2
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, costo: {cost:.4f}")
        # Aggiornamento dei fattori tramite gradient descent

        # Calcolo del gradiente
        U_grad = error.dot(V)
        V_grad = error.T.dot(U)
        
        U += alpha * U_grad
        V += alpha * V_grad

        losses.append(cost)
    
    return U, V

# Eseguiamo la fattorizzazione
k = 10         # ad esempio, 10 fattori latenti
epochs = 100   # numero di iterazioni
alpha = 0.001  # learning rate

U, V = unconstrained_matrix_factorization(R, mask, k=k, epochs=epochs, alpha=alpha)

# Una volta appresi U e V, la matrice dei rating completa è data da:
R_complete = U.dot(V.T)

# Ad esempio, per predire il rating (item_count) della sessione 0 sull'item con indice 5:
predicted_rating = R_complete[0, 5]
print(f"Predicted rating per sessione 0, item 5: {predicted_rating:.4f}")
